In [10]:
import pandas as pd 
import json 
import os 
import sys 

sys.path.append("..")
from settings import PROJECT_PATH, CLASSIFICATION_TARGET, REGRESSION_TARGET

### Chargement de la donnée 

In [2]:
transactions = pd.read_parquet(
    os.path.join(PROJECT_PATH, "transactions_post_feature_engineering.parquet")
)

transactions_extra_info = pd.read_parquet("../transactions_extra_infos.parquet")

with open("../features_used.json", "r") as f:
    feature_names = json.load(f)

with open("../categorical_features_used.json", "r") as f:
    categorical_features = json.load(f)

numerical_features = [col for col in feature_names if col not in categorical_features]



Afin de séparer la donnée en covid et post-covid, nous avons besoin de la date de transaction. Cette information se trouve dans le jeu de données transactions_extra_infos. Une jointure est alors nécessaire.

In [3]:
transactions = transactions.merge(
    transactions_extra_info[["id_transaction", "date_transaction"]],
    on="id_transaction",
    how="left",
)

transactions["anne_mois"] = transactions["date_transaction"].dt.to_period("M")

In [4]:
transactions_pre_covid = transactions[transactions["anne_mois"] < "2020-03"]

transactions_covid = transactions[
    transactions["anne_mois"].between("2020-03", "2021-07")
]
transactions_post_covid = transactions[transactions["anne_mois"] > "2021-07"]